[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/remidion/PGM-Project/blob/main/main.ipynb)

In [ ]:
# Imports
from getpass import getpass
import pandas as pd
import os

# Data Preparation

In [1]:
if os.path.exists('PGM-Project'):
  !git -C PGM-Project/ pull 
else:
  u = input('GitHub username: ')
  p = getpass('GitHub password: ')
  !git clone https://$u:$p@github.com/remidion/PGM-Project.git
  del u, p

Already up to date.


## Load toy data

In [2]:
planned, d_2016_10_21 = pd.read_excel("PGM-Project/data/toy.xlsx", sheet_name=[0,1]).values()

## Download MBTA Data

In [3]:
if not os.path.exists("MBTA Bus Ridership.geojson"):
  !wget https://opendata.arcgis.com/datasets/eec03d901d2e470ebd5758c60d793e8e_0.geojson
  !mv eec03d901d2e470ebd5758c60d793e8e_0.geojson MBTA Bus Ridership.geojson


--2020-11-17 01:45:39--  https://opendata.arcgis.com/datasets/eec03d901d2e470ebd5758c60d793e8e_0.geojson
Resolving opendata.arcgis.com (opendata.arcgis.com)... 35.169.194.0, 52.55.102.109, 34.205.105.40, ...
Connecting to opendata.arcgis.com (opendata.arcgis.com)|35.169.194.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/json]
Saving to: ‘eec03d901d2e470ebd5758c60d793e8e_0.geojson’

eec03d901d2e470ebd5     [            <=>     ]   1.29G  73.8MB/s    in 16s     

2020-11-17 01:45:56 (82.8 MB/s) - ‘eec03d901d2e470ebd5758c60d793e8e_0.geojson’ saved [1380620043]



In [4]:
if not os.path.exists("data"):
  !wget https://www.arcgis.com/sharing/rest/content/items/d685ba39d9a54d908f49a2a762a9eb47/data
if not os.path.exists("MBTA Bus Arrival Departure Aug-Sept 2018.csv"):
  !unzip data

if not os.path.exists("data.1")
  !wget https://www.arcgis.com/sharing/rest/content/items/1bd340b39942438685d8dcdfe3f26d1a/data
if not os.path.exists("MBTA Bus Arrival Departure Apr-June 2019.csv")
  !unzip data.1

--2020-11-17 01:45:56--  https://www.arcgis.com/sharing/rest/content/items/d685ba39d9a54d908f49a2a762a9eb47/data
Resolving www.arcgis.com (www.arcgis.com)... 18.234.22.28, 34.233.149.104, 18.234.22.251, ...
Connecting to www.arcgis.com (www.arcgis.com)|18.234.22.28|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ago-item-storage.s3.us-east-1.amazonaws.com/d685ba39d9a54d908f49a2a762a9eb47/MBTA_Bus_Arrival_Departure_Times_2018.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEKn%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJIMEYCIQDnn005fGlAG0WAPhxAzybd5yRdMj1xAvhs9noz7E6w7QIhAP0taqB30pIj%2BAwx3XE3TK2mfDFhShjF2WlRvVY52%2FmRKrQDCCEQABoMNjA0NzU4MTAyNjY1IgyTUHNrcKJkOqAs4LoqkQNUxvgbhcl8LPcrS6Z6ZJFuqYmIktXA2g%2BTjXhPBC1ZSV5htBHKFS7YPmcJIyHx1OOJkK1uOgJnwhALbT6OqWyRKxi%2BfEm2NDWVOaGrA4wQyjfcxCV%2FjIaL3My01r3S5B3JF9PcC%2BJs2HjvOB5XupBNjPHPqOwmXXcbn9h4KbZbUW4rmlveItmqilwYa6Clk%2BIMNXChCuE7UcEIZYwxIw2rDW4bOq8VXFx0nq6AzjhXsAYjT7XBySyXIHBocRCJr53NbX1iAHHvBU%2B6lD3GV8D

## Load MBTA Data

In [5]:
df = pd.read_json("MBTA_bus_ridership.geojson")

In [7]:
df2 = pd.read_csv("MBTA Bus Arrival Departure Apr-June 2019.csv")

In [8]:
df2

,service_date,route_id,direction,half_trip_id,stop_id,stop_name,stop_sequence,point_type,standard_type,scheduled,actual,scheduled_headway,headway
0,2019-04-01,01,Inbound,42976988.0,75,mit,4.0,Midpoint,Schedule,1900-01-01 05:19:00,1900-01-01 05:21:20,NaN,NaN
1,2019-04-01,01,Inbound,42976988.0,79,hynes,5.0,Midpoint,Schedule,1900-01-01 05:23:00,1900-01-01 05:24:17,NaN,NaN
2,2019-04-01,01,Inbound,42976988.0,187,masta,6.0,Midpoint,Schedule,1900-01-01 05:25:00,1900-01-01 05:26:05,NaN,NaN
3,2019-04-01,01,Inbound,42976988.0,59,Wasma,7.0,Midpoint,Schedule,1900-01-01 05:29:00,1900-01-01 05:27:59,NaN,NaN
4,2019-04-01,01,Inbound,42977170.0,110,hhgat,1.0,Startpoint,Headway,1900-01-01 05:30:00,1900-01-01 05:29:55,1200.0,1230.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7670643,2019-06-30,SL5,Outbound,44045095.0,15176,mawor,8.0,Midpoint,Headway,1900-01-02 00:54:00,NaN,900.0,NaN
7670644,2019-06-30,SL5,Outbound,44045095.0,55,Wasma,9.0,Midpoint,Headway,1900-01-02 00:55:00,NaN,900.0,NaN
7670645,2019-06-30,SL5,Outbound,44045095.0,60,Walen,10.0,Midpoint,Headway,1900-01-02 00:56:00,NaN,900.0,NaN
7670646,2019-06-30,SL5,Outbound,44045095.0,61,Melwa,11.0,Midpoint,Headway,1900-01-02 00:57:00,NaN,900.0,NaN


# Preprocess data

In [ ]:
planned.columns

In [ ]:
filter_rows = (planned['Ligne'] == 10.0) & \
              (planned['Direction'] == "Nord")

filter_columns = ['Ligne', 'De', 'Début', 'Fin', 'À', 'NoPermanent', 'Direction']

planned[filter_rows][filter_columns]

In [ ]:
filter_rows = (d_2016_10_21['ROUTE_ID'] == 10) & \
              (d_2016_10_21['DIRECTION'] == 1)

filter_columns = ['MEA_DATE', 'TRIP_ID', 'ROUTE_ID',
       'DIRECTION', 'STOP_ID', 'SCHEDULED_TIME',
       'MEA_ARR_TIME', 'MEA_DEP_TIME', 'NB_BOARDING', 'NB_DEBARKING',
       'PASSENGER_LOAD', 'AVG_PASSENGER_LOAD_TRIP', 'SYSRECNO']

d_2016_10_21[filter_rows][filter_columns]